In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jun 16 16:58:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.23       Driver Version: 511.23       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   47C    P8    25W / 170W |    519MiB / 12288MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 34.1 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

sklearn version:  0.24.2
TF version:  2.8.0
GPU installed:  True
1 Physical GPUs, 1 Logical GPUs


# DATA EDA

In [4]:
#byclass_train = pd.read_csv("/content/drive/MyDrive/emnist-byclass-train.csv")
byclass_test = pd.read_csv("C:/Users/user/Favorites/EMNIST_test_competition.csv")

#mapp = pd.read_csv("/content/drive/MyDrive/emnist-byclass-mapping.txt", delimiter = ' ', index_col=0, header=None, squeeze=True)

In [5]:
byclass_test.head()

,14,0,0.1,0.2,0.3,1,1.1,1.2,0.4,0.5,...,0.438,0.439,0.440,0.441,0.442,0.443,0.444,0.445,0.446,0.447
0,52,0,0,221,221,221,208,208,208,0,...,0,0,0,0,0,2,2,2,0,0
1,17,0,0,0,0,0,0,1,1,1,...,1,1,1,0,0,0,0,0,0,0
2,48,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10,0,0,0,0,0,0,0,0,0,...,84,84,84,127,127,127,255,255,255,254
4,49,1,4,4,4,2,2,2,0,0,...,1,0,0,0,2,2,2,0,0,0


# DATA Processing

In [6]:

x_test = byclass_test.iloc[:,1:]
y_test = byclass_test.iloc[:,0]

In [7]:
# Normalise data and reshape

x_test = np.array(x_test) / 255.0
y_test = np.array(y_test)


In [8]:

x_test.astype('float32')

array([[0.        , 0.        , 0.8666667 , ..., 0.00784314, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [9]:

x_test = x_test.reshape(x_test.shape[0], 28, 28)


In [10]:
#one-hot encoding
number_of_classes = 62


y_test = tf.keras.utils.to_categorical(y_test, number_of_classes)

In [11]:

x_test = x_test.reshape(-1, 28, 28, 1)


# Inference

In [12]:
import time
tf.random.set_seed(42)
np.random.seed(42)
new_model = tf.keras.models.load_model('byclass_vgg5.h5')

In [13]:
test_loss, test_acc = new_model.evaluate(x_test, y_test)

print("\n")
print("test_loss, test_acc: ",test_loss, test_acc)

4/4 [==============================] - 2s 40ms/step - loss: 13.8908 - accuracy: 0.2269


test_loss, test_acc:  13.890804290771484 0.22689075767993927


In [14]:
#빠른거!
tf.random.set_seed(42)
np.random.seed(42)
new_model = tf.keras.models.load_model('select_model_vgg5.h5')

In [15]:
test_loss, test_acc = new_model.evaluate(x_test, y_test)
print("\n")
print("test_loss, test_acc: ",test_loss, test_acc)

4/4 [==============================] - 0s 45ms/step - loss: 12.3873 - accuracy: 0.1933


test_loss, test_acc:  12.387263298034668 0.19327731430530548


In [ ]:
# start = time.time()
# terminate_time = time.time() -start

# hours, mu, ss = hms(terminate_time)
# print('inf time: ', hours, 'h', mu, 'm',ss,'s')